# Function Calling
One of the many ways to provide an LLM with real-time, dynamic data is to let it call functions. Or, to be more specifc, let it tell you to call a function for it and then provide the results so it can use it to fulfill a query.

In this notebook, I will show you how to implement function calling using the OpenAI API and GPT-4o-mini. We will create an analytics agent that can query data from a database to answer analytical questions we can ask in natural language!

I will use DuckDB as a light-weight in-memory database. This would work with any other database, of course!

### Loading Some Data
I found some product price index data set on Kaggle, which shows the historic price evolution of some foods in different US cities over the course of two decades. Let's load it to a table called product_prices.

In [92]:
import json

import duckdb

In [ ]:
duckdb.sql(
    """
    CREATE TABLE product_prices AS
    SELECT * FROM read_csv('ProductPriceIndex.csv');
    """
)

In [33]:
result = duckdb.sql(
    """
    SELECT * FROM product_prices LIMIT 5;
    """
)
print(result)

┌──────────────────┬────────────┬───────────┬───────────────┬───────────────┬──────────────────┬───────────────┬───────────────┐
│   productname    │    date    │ farmprice │ atlantaretail │ chicagoretail │ losangelesretail │ newyorkretail │ averagespread │
│     varchar      │    date    │  varchar  │    varchar    │    varchar    │     varchar      │    varchar    │    varchar    │
├──────────────────┼────────────┼───────────┼───────────────┼───────────────┼──────────────────┼───────────────┼───────────────┤
│ Strawberries     │ 2019-05-19 │ $1.16     │ $2.23         │ $1.70         │ $1.99            │ $2.54         │ 82.33%        │
│ Romaine Lettuce  │ 2019-05-19 │ $0.35     │ $1.72         │ $2.00         │ $1.69            │ $1.99         │ 428.57%       │
│ Red Leaf Lettuce │ 2019-05-19 │ $0.32     │ $1.84         │ $1.84         │ $1.69            │ $1.89         │ 467.19%       │
│ Potatoes         │ 2019-05-19 │ $1.50     │ $5.32         │ $5.14         │ $3.99            │ 

In [36]:
result = duckdb.sql(
    """
    SELECT min(date) AS min_date, max(date) AS max_date, COUNT(*) as num_records FROM product_prices;
    """
)
print(result)

┌────────────┬────────────┬─────────────┐
│  min_date  │  max_date  │ num_records │
│    date    │    date    │    int64    │
├────────────┼────────────┼─────────────┤
│ 1999-10-24 │ 2019-05-19 │       15766 │
└────────────┴────────────┴─────────────┘



In [44]:
result = duckdb.sql(
    """
    SELECT distinct(date) as dates FROM product_prices ORDER BY dates DESC LIMIT 10;
    """
)
print(result)

┌────────────┐
│   dates    │
│    date    │
├────────────┤
│ 2019-05-19 │
│ 2019-05-12 │
│ 2019-05-05 │
│ 2019-04-28 │
│ 2019-04-21 │
│ 2019-04-14 │
│ 2019-04-07 │
│ 2019-03-31 │
│ 2019-03-24 │
│ 2019-03-17 │
├────────────┤
│  10 rows   │
└────────────┘



We have a column called productname, a date column and a bunch of columns with price information for different cities. The information ranges from 1999 to 2019 and seems to give us an overview of price evolution for different products on a weekly basis. 

Let's assume this is a production database that changes over time. Products and new price information can be added. The LLM will need to be able to access this information on-demand. We will give it the capabilities to call functions that run queries and return the results for it to use to answer questions.

### Creating Query Functions

First, let's create a decorator function called *result_to_json* to get the DuckDB query results in JSON format so we can pass them to the LLM as text.

In [113]:
from typing import Callable

from duckdb.duckdb import DuckDBPyRelation

def result_to_json(func: Callable[..., DuckDBPyRelation]) -> Callable[..., str]:
    def inner(*args, **kwargs) -> str:
        result = func(*args, **kwargs)
        return result.fetchdf().to_json(orient="records", date_format="iso")
    return inner

Then, I will create functions to do the following things:
- Get sample data
- Get all distinct products in the database
- Current the most recent prices for a specific product
- All records for a product in a date range

Of course, in a real-world scenario we would have to pinpoint these capabilities to match business needs and frequent questions. For now, let's assume these are important informations that are requested frequently.

In [115]:
@result_to_json
def get_data_sample() -> DuckDBPyRelation:
    return duckdb.sql(
        """
        SELECT * FROM product_prices LIMIT 5;
        """
    )

@result_to_json
def get_distinct_products() -> DuckDBPyRelation:
    return duckdb.sql(
        """
        SELECT distinct(productname) FROM product_prices;
        """
    )

@result_to_json
def get_current_prices(product_name: str) -> DuckDBPyRelation:
    return duckdb.sql(
        f"""
        SELECT * 
        FROM product_prices
        WHERE productname = \'{product_name}\'
        AND date = (
            SELECT MAX(date)
            FROM product_prices
            WHERE productname = '{product_name}'
        )
        """
    )

@result_to_json
def get_records_in_date_range(product_name: str, min_date: str, max_date: str):
    return duckdb.sql(
        f"""
        SELECT *
        FROM product_prices
        WHERE productname = \'{product_name}\'
        AND date >= \'{min_date}\'
        AND date <= \'{max_date}\'
        """
)

### Creating a Chatbot Using the OpenAI API
So far, so good. Now, let's create a simple chatbot that we can interact with which uses the OpenAI API for LLM integration. We will later integrate the above function calls with the LLM, making it a little data analysis agent.

We start by crafting our system prompt to tell the LLM its purpose. In the system prompt, I will use the outputs  of the *get_sample_data* and *get_distinct_products* functions so the LLM knows about the data format and which products it can answer questions for. We will also tell the LLM in the system prompt that today is the max. date of our dataset, 2019–05–19, so we can treat the data as if it is up to date.

In [81]:
system_prompt = f"""
You are a helpful assistant that analyzes data. You will answer questions about the price development of different food products based
on data stored in a database. 

Here is a sample of what the data that you will analyze looks like:
{get_data_sample()}

The database contains information about the following distinct products: 
{get_distinct_products()}

Today's date is 2019-05-19. Data is available from 1999-10-24 til 2019-05-19. 

Only answer questions related to analyzing the product price data. If the user requests you to do anything else, 
kindly tell them that you are here to analyze product price data and that you cannot help with other requests.
"""

In [82]:
print(system_prompt)


You are a helpful assistant that analyzes data. You will answer questions about the price development of different food products based
on data stored in a database. 

Here is a sample of what the data that you will analyze looks like:
[{"productname":"Strawberries","date":"2019-05-19T00:00:00.000","farmprice":"$1.16","atlantaretail":"$2.23","chicagoretail":"$1.70","losangelesretail":"$1.99","newyorkretail":"$2.54","averagespread":"82.33%"},{"productname":"Romaine Lettuce","date":"2019-05-19T00:00:00.000","farmprice":"$0.35","atlantaretail":"$1.72","chicagoretail":"$2.00","losangelesretail":"$1.69","newyorkretail":"$1.99","averagespread":"428.57%"},{"productname":"Red Leaf Lettuce","date":"2019-05-19T00:00:00.000","farmprice":"$0.32","atlantaretail":"$1.84","chicagoretail":"$1.84","losangelesretail":"$1.69","newyorkretail":"$1.89","averagespread":"467.19%"},{"productname":"Potatoes","date":"2019-05-19T00:00:00.000","farmprice":"$1.50","atlantaretail":"$5.32","chicagoretail":"$5.14","lo

To make the user prompts dynamic, we will leverage a Gradio chat UI that dynamically passes the user input, the previous conversation, as well as the system prompt to the OpenAI model.

The Gradio ChatInterface object expects us to pass a callback method that takes exactly two arguments: *message* and *history*. The first one is the new user input message, while the history is the whole previous conversation between the user and the LLM. For more information, you can check the Gradio docs here: https://www.gradio.app/docs/gradio/chatinterface

For now, let's create the callback method for the chat interface and launch the Gradio UI. In the callback method, we will use the latest user message, the message history and the system prompt to build our *messages* API input for the GPT-4o-mini model. Like always, we craft our model input passing a list of JSON objects that looks as follows:

To allow the model to see the full previous conversation, we will extend that list with additional objects. For each LLM response, we will add an object that uses the role *assistant*. This feature can be used to pass the full previous conversation, but also to pass responses from other agents that assist fulfilling a user query. For every new user message, we will append an object that uses the role *user*.

Over time, the messages input for the OpenAI API will look something like this:

By passing the full conversation to the API every time, it seems like the LLM stores a state about what was said before. In reality, it is fed the full conversation for every inference, making it seem like the model has a memory of the past.

Let's create a *chat_callback* function that takes *new_message* and *message_history* as inputs from the Gradio UI callback and then builds the *messages* input for the OpenAI API. We  will then use it to call the API and generate a response, which we output in the Gradio UI. Just like that, we built an LLM-backed chatbot! We will make it access our database after that.

In [122]:
import os

from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

openai = OpenAI()

def chat_callback(new_message: str, message_history: list[tuple[str, str]]):
    messages = [{"role": "system", "content": system_prompt}]
    for user, assistant in message_history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": new_message})
    return call_openai(messages)

def call_openai(messages: list[str]) -> str:
    response = openai.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages
    )
    return response.choices[0].message.content

gr.ChatInterface(fn=chat_callback).launch()

Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


/Users/kevinpilch/Library/Caches/pypoetry/virtualenvs/llm-usecases-n3dvwjYK-py3.12/lib/python3.12/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.1, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(
Traceback (most recent call last):
  File "/Users/kevinpilch/Library/Caches/pypoetry/virtualenvs/llm-usecases-n3dvwjYK-py3.12/lib/python3.12/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kevinpilch/Library/Caches/pypoetry/virtualenvs/llm-usecases-n3dvwjYK-py3.12/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kevinpilch/Library/Caches/pypoetry/virtualenvs/llm-usecases-n3dvwjYK-py3.12/lib/python3.12/site-packages/gradio/blocks.py", line 1

We now have a real chat that integrates with OpenAI! Unfortunately, it does not yet know much about the prices in the database. It only knows about the products and sample records from the system prompt. Let's change that!

### Enabling Function Calling to Let LLM Query Database
The OpenAI chat API gives us the option to pass a list of tools the model can use. We will have to provide the tool information in a specific format, just like we did with the messages. The model then decides when it wants to use a tool to answer a user query and returns a flag to let us know to call a function. We then call the function ourselves and provide the function output to the LLM in a prompt using the role *tool*. The model will then use that output to answer the user question. You can read the function calling docs here: https://platform.openai.com/docs/guides/function-calling.

Let's go step by step.

First, we need to describe our functions using a special format the model has been trained on.

In [124]:
current_prices_function = {
    "name": "get_current_prices",
    "description": "Gets the most recent prices for a specific product. Call this function when a user asks for the current price of a product.",
    "parameters": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The product the user wants to know the price for. Make sure to format the value according to the list of distinct products",
            },
        },
        "required": ["product_name"],
        "additionalProperties": False
    }
}

prices_in_date_range_function = {
    "name": "get_records_in_date_range",
    "description": """
    Gets the prices for a specific product in a date range. Call this when the user asks questions that require information from a larger 
    time frame such as 'What was the lowest price for potatoes last year?' or 'On average, where are organges the most expensive historically?'.
    """,
    "parameters": {
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "The product the user searches information for. Make sure to format the value according to the list of distinct products",
            },
            "min_date": {
                "type": "string",
                "description": "The lower boundary of the date range. Make sure to pass ISO 8601 date formats, such as 2019-05-19.",
            },
            "max_date": {
                "type": "string",
                "description": "The upper boundary of the date range. Make sure to pass ISO 8601 date formats, such as 2019-05-19.",
            },
        },
        "required": ["product_name", "min_date", "max_date"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": current_prices_function},
    {"type": "function", "function": prices_in_date_range_function},
]

In the API call, we can pass the tools list along, allowing the model to use the tools. If it decides to use a tool to answer a question, it will return a special finish_reason value in the response which we have to check for: 

`response.choices[0].finish_reason=="tool_calls"`

If the condition is true, we will have to call a function for the model. It will also provide the correct input arguments, which we will have to extract. Once we called the function with the provided arguments, we pass it the result in the prompt using the role *tools*. The model will use that result to finally answer the user query.

Let's rework the *chat_callback* and *call_openai* functions a bit. We will also create a *call_tool* function to handle the tool calls requested by the model.

In [125]:
def call_openai(messages: list[str], tools: list[str] = None) -> str:
    if tools is not None:
        return openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
    return openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

def chat_callback(new_message: str, message_history: list[tuple[str, str]]):
    messages = [{"role": "system", "content": system_prompt}]
    for user, assistant in message_history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": new_message})
    response = call_openai(messages, tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = call_tool(message.tool_calls[0])
        messages.append(message)
        messages.append(response)
        response = call_openai(messages) # Call API again with the function output without providing tools because we would have to implement more complex handling 
    
    return response.choices[0].message.content
 

def call_tool(tool_call: dict):
    """
    This function checks what function to call for the model, extracts the arguments, then calls the function and returns the response
    in the format required by the OpenAI API.
    """
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    if function_name == "get_current_prices":
        product_name = arguments.get("product_name")
        return { 
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(
                {
                    "product_name": product_name, 
                    "current_prices": get_current_prices(product_name)
                }
            ),
        }
    elif function_name == "get_records_in_date_range":
        product_name = arguments.get("product_name")
        min_date = arguments.get("min_date")
        max_date = arguments.get("max_date")
        return {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(
                {
                    "product_name": product_name,
                    "min_date": min_date,
                    "max_date": max_date,
                    "prices": get_records_in_date_range(
                        product_name = product_name,
                        min_date = min_date,
                        max_date = max_date
                    )
                }
            )
        }
    else: 
        raise Exception(f"Unknown function {function_name}")
    

gr.ChatInterface(fn=chat_callback).launch()

Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


/Users/kevinpilch/Library/Caches/pypoetry/virtualenvs/llm-usecases-n3dvwjYK-py3.12/lib/python3.12/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.1, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


Awesome! We can now chat with out data. 

In a real-world scenario, we would probably have to add more functions. We would also have to consider security and data access limitations based on the user role, testing, deployment, rate-limits etc.

For a quick POC, this is pretty neat though, don't you think?

Also, keep in mind the heavy lifting the model does for us. It even formats query parameters correctly based on our system prompts and function description. This once again shows how powerful these models really are if you wrap them around some system as an abstraction layer. They can make complex tools more easily available to a broader audience.